# Open-Meteo : Weather Forecast API

Open-Meteo features weather forecasts aswell as historical weather data, with up to 1km resolution.

We'll use the free available licence for non-commercial use, which restricts its usage to less than 10 000 daily requests.

Forecast documentation : https://open-meteo.com/en/docs

Historical Data Documentation : https://open-meteo.com/en/docs/historical-weather-api

# Imports

In [1]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

from geopy.geocoders import Nominatim
import numpy as np

# Documentation

Weather Variables : WMO Weather interpretation codes (WW)

In [2]:
wmo_codes = pd.DataFrame({"code": [0, 1, 2, 3, 45, 48, 51, 53, 55, 56, 57, 61, 63, 65, 66, 67, 71, 73, 75, 77, 80, 81, 82, 85, 86, 95, 96, 99]})
wmo_codes['description'] = np.select(
    [wmo_codes['code'].isin([0]),
     wmo_codes['code'].isin([1, 2, 3]),
     wmo_codes['code'].isin([45, 48]),
     wmo_codes['code'].isin([51, 53, 55]),
     wmo_codes['code'].isin([56, 57]),
     wmo_codes['code'].isin([61, 63, 65]),
     wmo_codes['code'].isin([66, 67]),
     wmo_codes['code'].isin([71, 73, 75]),
     wmo_codes['code'].isin([77]),
     wmo_codes['code'].isin([80, 81, 82]),
     wmo_codes['code'].isin([85, 86]),
     wmo_codes['code'].isin([95]),
     wmo_codes['code'].isin([96, 99]),
    ],
    ['Clear sky',
     'Mainly clear, partly cloudy, and overcast',
     'Fog and depositing rime fog',
     'Drizzle: Light, moderate, and dense intensity',
     'Freezing Drizzle: Light and dense intensity',
     'Rain: Slight, moderate and heavy intensity',
     'Freezing Rain: Light and heavy intensity',
     'Snow fall: Slight, moderate, and heavy intensity',
     'Snow grains',
     'Rain showers: Slight, moderate, and violent',
     'Snow showers slight and heavy',
     'Thunderstorm: Slight or moderate',
     'Thunderstorm with slight and heavy hail'
    ]
)
print("WMO weather codes :")
display(wmo_codes)

WMO weather codes :


,code,description
0,0,Clear sky
1,1,"Mainly clear, partly cloudy, and overcast"
2,2,"Mainly clear, partly cloudy, and overcast"
3,3,"Mainly clear, partly cloudy, and overcast"
4,45,Fog and depositing rime fog
5,48,Fog and depositing rime fog
6,51,"Drizzle: Light, moderate, and dense intensity"
7,53,"Drizzle: Light, moderate, and dense intensity"
8,55,"Drizzle: Light, moderate, and dense intensity"
9,56,Freezing Drizzle: Light and dense intensity


# Simple requests

## Documentation's example usecase

Getting the weather forecasts at ESME's Ivry-Sur-Seine campus.

In [3]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after=3600)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": 48.81452162820077,
	"longitude": 2.3948078406657114,
	"hourly": ["temperature_2m", 
            "relative_humidity_2m", 
            "wind_speed_10m",
            "wind_direction_10m",
            "apparent_temperature",
            "cloud_cover",
            "precipitation", 
            "weather_code",
            "is_day"
            ],
    "timezone": "auto"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]

Quick check of the selected address

In [4]:
geolocator = Nominatim(user_agent="ESMEAirPollutionPrediction")
location = geolocator.reverse(f"{params['latitude']}, {params['longitude']}")
print(location.address)

60, Boulevard de Brandebourg, Ivry Port, Ivry-sur-Seine, L'Haÿ-les-Roses, Val-de-Marne, Île-de-France, France métropolitaine, 94200, France


In [5]:
print(f"Coordinates {response.Latitude()}°E {response.Longitude()}°N")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()

hourly_data = {
    "date": pd.date_range(
        start=pd.to_datetime(hourly.Time(), unit = "s"),
        end=pd.to_datetime(hourly.TimeEnd(), unit = "s"),
        freq=pd.Timedelta(seconds=hourly.Interval()),
        inclusive="left"
        )
    }
for i in range(hourly.VariablesLength()):  # Iterating through all the variables in params["hourly"]
    hourly_data[params["hourly"][i]] = hourly.Variables(i).ValuesAsNumpy()

hourly_dataframe = pd.DataFrame(data=hourly_data)
display(hourly_dataframe)

Coordinates 48.81999969482422°E 2.3999996185302734°N
Elevation 36.0 m asl
Timezone b'Europe/Paris' b'CET'
Timezone difference to GMT+0 3600 s


,date,temperature_2m,relative_humidity_2m,wind_speed_10m,wind_direction_10m,apparent_temperature,cloud_cover,precipitation,weather_code,is_day
0,2023-11-15 23:00:00,8.156500,94.0,3.600000,180.000000,6.923271,99.0,0.0,3.0,0.0
1,2023-11-16 00:00:00,8.156500,95.0,3.096837,144.462234,7.034328,98.0,0.0,3.0,0.0
2,2023-11-16 01:00:00,8.356501,95.0,3.563818,135.000107,7.215137,100.0,0.0,3.0,0.0
3,2023-11-16 02:00:00,8.556500,93.0,3.219938,153.435013,7.437356,99.0,0.0,3.0,0.0
4,2023-11-16 03:00:00,8.406500,96.0,4.610250,141.340164,7.163174,100.0,0.0,3.0,0.0
...,...,...,...,...,...,...,...,...,...,...
163,2023-11-22 18:00:00,5.399500,84.0,3.545589,246.037506,3.251421,40.0,0.0,1.0,0.0
164,2023-11-22 19:00:00,5.149500,86.0,4.072935,224.999893,2.940664,58.0,0.0,3.0,0.0
165,2023-11-22 20:00:00,5.099500,87.0,4.510787,208.610367,2.848188,77.0,0.0,3.0,0.0
166,2023-11-22 21:00:00,5.199500,88.0,5.154415,204.775116,2.903629,95.0,0.0,3.0,0.0


matching the weather codes to the wmo df :

In [6]:
if not "description" in hourly_dataframe.columns: 
    hourly_dataframe = hourly_dataframe.merge(wmo_codes, left_on="weather_code", right_on="code").drop("code", axis=1)
print("weather forecast at", location.address)
display(hourly_dataframe)

weather forecast at 60, Boulevard de Brandebourg, Ivry Port, Ivry-sur-Seine, L'Haÿ-les-Roses, Val-de-Marne, Île-de-France, France métropolitaine, 94200, France


,date,temperature_2m,relative_humidity_2m,wind_speed_10m,wind_direction_10m,apparent_temperature,cloud_cover,precipitation,weather_code,is_day,description
0,2023-11-15 23:00:00,8.156500,94.0,3.600000,180.000000,6.923271,99.0,0.0,3.0,0.0,"Mainly clear, partly cloudy, and overcast"
1,2023-11-16 00:00:00,8.156500,95.0,3.096837,144.462234,7.034328,98.0,0.0,3.0,0.0,"Mainly clear, partly cloudy, and overcast"
2,2023-11-16 01:00:00,8.356501,95.0,3.563818,135.000107,7.215137,100.0,0.0,3.0,0.0,"Mainly clear, partly cloudy, and overcast"
3,2023-11-16 02:00:00,8.556500,93.0,3.219938,153.435013,7.437356,99.0,0.0,3.0,0.0,"Mainly clear, partly cloudy, and overcast"
4,2023-11-16 03:00:00,8.406500,96.0,4.610250,141.340164,7.163174,100.0,0.0,3.0,0.0,"Mainly clear, partly cloudy, and overcast"
...,...,...,...,...,...,...,...,...,...,...,...
163,2023-11-19 09:00:00,12.236501,78.0,16.087610,229.537857,9.500156,40.0,0.0,1.0,1.0,"Mainly clear, partly cloudy, and overcast"
164,2023-11-22 16:00:00,6.899500,74.0,5.600286,315.000092,4.391726,79.0,0.0,1.0,1.0,"Mainly clear, partly cloudy, and overcast"
165,2023-11-22 17:00:00,6.049500,79.0,3.758510,286.699310,3.827928,59.0,0.0,1.0,0.0,"Mainly clear, partly cloudy, and overcast"
166,2023-11-22 18:00:00,5.399500,84.0,3.545589,246.037506,3.251421,40.0,0.0,1.0,0.0,"Mainly clear, partly cloudy, and overcast"


In [18]:
hourly_dataframe.to_csv(f"../data/forecast_{str(hourly_dataframe.at[0, 'date']).split()[0]}.csv")  # saved to check later and compare it to real life values

Utiliser directions et vitesses de vents

comparer forecasts au réel

voir meteofrance directement ?
